# Predicción de la producción de activos industriales utilizando resultados de detección de anomalías con Elastic

In [16]:
import pandas as pd
import numpy as np

from datetime import datetime
from datetime import date
from elasticsearch import Elasticsearch

from sklearn.metrics import mean_squared_error


### 1. Leer datos 

In [17]:
anomalias=pd.read_csv("Output/df_outliers_Elastic_bucketspan_1d_p.csv", parse_dates=['date'])
print(anomalias.shape)                      
anomalias.head()

(802, 6)


,asset_id,date,outlier_t1,outlier_t2,outlier_t3,outlier_t4
0,2,2018-10-26,NaN,NaN,NaN,77.24560
1,2,2018-10-27,NaN,NaN,NaN,80.20610
2,2,2019-01-11,NaN,NaN,NaN,77.91047
3,2,2019-01-15,NaN,NaN,NaN,76.13505
4,2,2019-01-20,NaN,NaN,NaN,78.59278


In [18]:
anomalias.describe()

,asset_id,outlier_t1,outlier_t2,outlier_t3,outlier_t4
count,802.000000,249.000000,228.000000,264.000000,243.000000
mean,41.907731,87.250372,88.771262,86.300002,84.898008
std,23.757650,6.316665,6.208445,6.448459,6.966076
min,2.000000,75.151870,75.154850,75.111540,75.073080
25%,23.000000,82.322280,84.196947,81.250390,78.793445
50%,40.000000,86.946042,88.967740,86.354930,84.383820
75%,61.000000,91.576020,93.867410,91.207960,90.773420
max,83.000000,99.845733,99.930953,99.549723,98.597267


In [19]:
outlier=anomalias.filter(regex='out')
for col in outlier.columns:
    anomalias[col]=np.where(anomalias[col]>=75,1,0)
anomalias

,asset_id,date,outlier_t1,outlier_t2,outlier_t3,outlier_t4
0,2,2018-10-26,0,0,0,1
1,2,2018-10-27,0,0,0,1
2,2,2019-01-11,0,0,0,1
3,2,2019-01-15,0,0,0,1
4,2,2019-01-20,0,0,0,1
...,...,...,...,...,...,...
797,83,2019-03-17,1,0,0,0
798,83,2019-03-18,1,0,0,0
799,83,2019-03-19,1,0,0,0
800,83,2019-07-25,1,1,0,0


In [20]:
Y_train=pd.read_csv(('Datos/train_y.csv'))
Y_train.head()

,SAMPLE_ID,PRODUCTION_GROUP_2,PRODUCTION_GROUP_3
0,1,3850220.0,9019860.320
1,2,3793240.0,8568867.304
2,3,3870570.0,8549258.912
3,4,3801380.0,8451216.952
4,5,3752540.0,8323762.404


In [21]:
X_train=pd.read_csv("Output/X_train_formateado.csv", parse_dates=['date'])
X_train

,asset_id,group_id,week,weekday,t1,t2,t3,t4,total_t,week_day,date,day
0,1,2,1,1,NaN,NaN,19.0,0.0,2,1_1,2018-01-01,1
1,1,2,1,2,NaN,NaN,8.0,0.0,2,1_2,2018-01-02,2
2,1,2,1,3,NaN,NaN,9.0,0.0,2,1_3,2018-01-03,3
3,1,2,1,4,NaN,NaN,6.0,0.0,2,1_4,2018-01-04,4
4,1,2,1,5,NaN,NaN,6.0,2.0,2,1_5,2018-01-05,5
...,...,...,...,...,...,...,...,...,...,...,...,...
60419,83,3,104,3,1.0,0.0,48.0,8.0,4,104_3,2019-12-25,724
60420,83,3,104,4,0.0,0.0,37.0,1.0,4,104_4,2019-12-26,725
60421,83,3,104,5,1.0,0.0,48.0,9.0,4,104_5,2019-12-27,726
60422,83,3,104,6,2.0,0.0,53.0,11.0,4,104_6,2019-12-28,727


In [22]:
#Leer datos de capacidad nominal por asset por semana

capacity=pd.read_csv('Datos/assets.csv', header=0, names=['asset_id', 'nominalc_week'],
                     dtype={'asset_id':int, 'nominalc_week':float} )

#agregar columna de capacidad por día
capacity['nominalc_day']=capacity['nominalc_week']/7     

#unir capcity a df data
data=X_train.merge(capacity[['asset_id','nominalc_day']])
data.head()


,asset_id,group_id,week,weekday,t1,t2,t3,t4,total_t,week_day,date,day,nominalc_day
0,1,2,1,1,NaN,NaN,19.0,0.0,2,1_1,2018-01-01,1,18857.142857
1,1,2,1,2,NaN,NaN,8.0,0.0,2,1_2,2018-01-02,2,18857.142857
2,1,2,1,3,NaN,NaN,9.0,0.0,2,1_3,2018-01-03,3,18857.142857
3,1,2,1,4,NaN,NaN,6.0,0.0,2,1_4,2018-01-04,4,18857.142857
4,1,2,1,5,NaN,NaN,6.0,2.0,2,1_5,2018-01-05,5,18857.142857


## 2. Unir tabla anomalías con X_train

In [23]:
anomalias['date']=anomalias['date'].astype(str)
data['date']=data['date'].astype(str)

In [24]:
assets_con_anomalias=anomalias['asset_id'].unique()
assets_con_anomalias

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 14, 16, 17, 18, 19, 20, 22, 23,
       24, 25, 26, 28, 29, 30, 31, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       45, 46, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
       65, 67, 68, 69, 70, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83],
      dtype=int64)

In [25]:
todos_los_assets=list(range(1,84))
assets_sin_anomalias=np.setdiff1d(todos_los_assets,assets_con_anomalias)

assets_sin_anomalias

array([ 1, 11, 12, 13, 15, 21, 27, 32, 33, 44, 47, 48, 49, 66, 71, 72])

In [26]:
#Groupby de ambos dataframe por asset
anomalias=anomalias.set_index('asset_id')
anom_porAsset=anomalias.groupby(anomalias.index)

data=data.set_index('asset_id')
porAsset=data.groupby(data.index)

In [27]:
%%time
new_data={}
for c in assets_con_anomalias:
    outlier=anom_porAsset.get_group(c)
    asset  =     porAsset.get_group(c).reset_index()
    new_data[c]=pd.merge(asset,outlier, how='left', on=['date'])
for s in assets_sin_anomalias:
    new_data[s]=porAsset.get_group(s).reset_index()

Wall time: 330 ms


In [28]:
new_data[2]

,asset_id,group_id,week,weekday,t1,t2,t3,t4,total_t,week_day,date,day,nominalc_day,outlier_t1,outlier_t2,outlier_t3,outlier_t4
0,2,2,1,1,NaN,NaN,7.0,0.0,2,1_1,2018-01-01,1,13285.714286,NaN,NaN,NaN,NaN
1,2,2,1,2,NaN,NaN,15.0,2.0,2,1_2,2018-01-02,2,13285.714286,NaN,NaN,NaN,NaN
2,2,2,1,3,NaN,NaN,9.0,0.0,2,1_3,2018-01-03,3,13285.714286,NaN,NaN,NaN,NaN
3,2,2,1,4,NaN,NaN,12.0,1.0,2,1_4,2018-01-04,4,13285.714286,NaN,NaN,NaN,NaN
4,2,2,1,5,NaN,NaN,8.0,1.0,2,1_5,2018-01-05,5,13285.714286,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,2,2,104,3,NaN,NaN,9.0,0.0,2,104_3,2019-12-25,724,13285.714286,NaN,NaN,NaN,NaN
724,2,2,104,4,NaN,NaN,7.0,0.0,2,104_4,2019-12-26,725,13285.714286,NaN,NaN,NaN,NaN
725,2,2,104,5,NaN,NaN,7.0,0.0,2,104_5,2019-12-27,726,13285.714286,NaN,NaN,NaN,NaN
726,2,2,104,6,NaN,NaN,10.0,0.0,2,104_6,2019-12-28,727,13285.714286,NaN,NaN,NaN,NaN


### 3. Definición de función obtener error y y_predict

In [31]:
def resultados(dict_df):
    '''FUNCION PARA CALCULAR EL ERROR ASIGNANDO EL VALOR DE PRODUCCION NOMINAL DIARIO A LOS DÍAS EN QUE NO HUBO 
    NINGUNA ANOMALÍAS Y 0 EN CASO CONTRARIO 
    
    dict_df: diccionario resultante con la detección de anomalías con un dataframe por cada asset

    '''
    
    X=pd.concat(dict_df.values())
    X=X.reset_index(drop=True)
    
    X=X[['asset_id', 'group_id', 'week', 'weekday','total_t','nominalc_day', 'outlier_t1','outlier_t2','outlier_t3','outlier_t4']]
    

    X['produccion']=np.where(
        ( ( X['outlier_t1']==1 ) | ( X['outlier_t2']==1 ) | (X['outlier_t3']==1)  | ( X['outlier_t4']==1 ) ),
        0, X['nominalc_day'] )
   
    #Agregamos los datos por grupo y semana para obtener Y_predicted

    ProduccionGrupoSemana=X.groupby(['group_id','week'])
    Y_predicted=ProduccionGrupoSemana.agg({'produccion':'sum'})
    Y_predicted=Y_predicted.reset_index()

    # Configuramos el formato  Y_predicted para tener el mismo que en Y_train
    Y_predicted=pd.pivot_table(Y_predicted,index=['week'], columns=['group_id'], values=['produccion'])
    Y_predicted=Y_predicted.reset_index()
    Y_predicted.columns.set_levels(['production_group_2', 'production_group_3', ''], level=1, inplace=True)
    Y_predicted.columns=list(map("".join, Y_predicted))
    Y_predicted.columns=Y_train.columns

    # Calculamos el error para cada guropo
    MSE2=mean_squared_error(Y_train.PRODUCTION_GROUP_2, Y_predicted.PRODUCTION_GROUP_2)
    MSE3=mean_squared_error(Y_train.PRODUCTION_GROUP_3, Y_predicted.PRODUCTION_GROUP_3)

    
    #Calculamos el error total
    Y_train_tot = Y_train.PRODUCTION_GROUP_2.to_list() + Y_train.PRODUCTION_GROUP_3.to_list()
    Y_predicted_tot = Y_predicted.PRODUCTION_GROUP_2.to_list() +  Y_predicted.PRODUCTION_GROUP_3.to_list() 
    ErrorTotal = mean_squared_error(Y_train_tot, Y_predicted_tot)
    

    return Y_predicted, ErrorTotal

### 4. Resultados

In [32]:
y_predicted, error= resultados(new_data)
print('error =', np.format_float_scientific(error))
y_predicted

error = 2.6425307652375458e+11


,SAMPLE_ID,PRODUCTION_GROUP_2,PRODUCTION_GROUP_3
0,1,4.070000e+06,9.804196e+06
1,2,4.070000e+06,9.804196e+06
2,3,4.070000e+06,9.804196e+06
3,4,4.070000e+06,9.804196e+06
4,5,4.061771e+06,9.545767e+06
...,...,...,...
99,100,4.070000e+06,9.761339e+06
100,101,4.035914e+06,9.652339e+06
101,102,4.070000e+06,9.588839e+06
102,103,4.070000e+06,9.696125e+06
